In [1]:
import numpy as np# linear algebra 
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv) 
import requests 
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://understat.com/league'
leagues = ['La_Liga', 'Bundesliga', 'EPL', 'Serie_A', 'Ligue_1']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

In [6]:
url = base_url+'/'+leagues[0]+'/'+seasons[9] 
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")
scripts = soup.find_all('script')

In [8]:
import json
string_with_json_obj = ''
for el in scripts:
    if 'teamsData' in el.text:
        string_with_json_obj = el.text.strip()

In [19]:
# usunięcie symboli

ind_start = string_with_json_obj.index("('")+2 
ind_end = string_with_json_obj.index("')") 
json_data = string_with_json_obj[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#print(string_with_json_obj)

In [20]:
data = json.loads(json_data)

In [23]:
# You can convert it into a DataFrame using pandas
df1 = pd.DataFrame(data.values())

# Print or further process the DataFrame
print(df1)

     id            title                                            history
0   138          Sevilla  [{'h_a': 'h', 'xG': 0.714148, 'xGA': 1.06796, ...
1   140    Real Sociedad  [{'h_a': 'h', 'xG': 1.46019, 'xGA': 1.27791, '...
2   142           Getafe  [{'h_a': 'h', 'xG': 0.804505, 'xGA': 1.67817, ...
3   143  Atletico Madrid  [{'h_a': 'h', 'xG': 3.01246, 'xGA': 1.13452, '...
4   144       Las Palmas  [{'h_a': 'h', 'xG': 0.871699, 'xGA': 1.49411, ...
5   145   Rayo Vallecano  [{'h_a': 'a', 'xG': 2.09532, 'xGA': 1.1196, 'n...
6   146         Valencia  [{'h_a': 'a', 'xG': 1.06796, 'xGA': 0.714148, ...
7   147    Athletic Club  [{'h_a': 'h', 'xG': 0.700577, 'xGA': 1.33266, ...
8   148        Barcelona  [{'h_a': 'a', 'xG': 1.67817, 'xGA': 0.804505, ...
9   150      Real Madrid  [{'h_a': 'a', 'xG': 1.33266, 'xGA': 0.700577, ...
10  152       Celta Vigo  [{'h_a': 'h', 'xG': 0.783184, 'xGA': 0.964645,...
11  153       Real Betis  [{'h_a': 'a', 'xG': 0.969349, 'xGA': 0.737812,...
12  154     

In [21]:
teams = {}
for id in data.keys():
    teams[id] = data[id]['title']
    

In [22]:
columns = []
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

In [24]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Las Palmas.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Granada.
Added data for Osasuna.
Added data for Alaves.
Added data for Almeria.
Added data for Girona.
Added data for Mallorca.
Added data for Cadiz.


In [27]:
for team, df in dataframes.items(): 
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

In [28]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [29]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

In [30]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

In [32]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [33]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

In [34]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
full_stat.columns = ['#', 'team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xG_diff', 'NPxG', 'xGA', 'xGA_diff', 'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'xPTS_diff']
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,#,team,M,W,D,L,G,GA,PTS,xG,...,xGA,xGA_diff,NPxGA,NPxGD,PPDA,OPPDA,DC,ODC,xPTS,xPTS_diff
0,1,Real Madrid,14,11,2,1,31,9,35,30.61,...,17.99,8.99,17.99,11.07,12.81,16.90,142,53,26.48,-8.52
1,2,Girona,14,11,2,1,32,17,35,28.95,...,21.65,4.65,20.17,8.04,9.83,12.96,106,76,23.84,-11.16
2,3,Atletico Madrid,13,10,1,2,30,12,31,28.38,...,12.06,0.06,12.06,14.83,14.77,16.66,127,64,27.90,-3.10
3,4,Barcelona,14,9,4,1,27,14,31,28.75,...,15.03,1.03,15.03,12.23,8.03,16.57,135,86,28.61,-2.39
4,5,Athletic Club,14,7,4,3,26,18,25,28.25,...,17.69,-0.31,16.94,9.07,10.17,10.74,90,68,24.11,-0.89
5,6,Real Sociedad,14,7,4,3,25,17,25,20.85,...,19.71,2.71,18.23,0.39,7.95,12.96,68,64,20.23,-4.77
6,7,Real Betis,14,6,6,2,18,16,24,20.92,...,16.78,0.78,15.29,5.62,12.49,9.85,113,77,22.61,-1.39
7,8,Getafe,14,4,7,3,17,18,19,17.61,...,20.37,2.37,18.14,-2.02,10.83,7.46,53,87,17.41,-1.59
8,9,Rayo Vallecano,14,4,7,3,16,18,19,18.51,...,20.81,2.81,19.32,-4.53,9.92,7.43,71,126,18.57,-0.43
9,10,Valencia,14,5,4,5,16,18,19,17.08,...,16.64,-1.36,15.16,0.43,11.15,10.91,49,66,19.95,0.95


In [35]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2018':      #             team   M   W  D   L   G  GA  PTS    xG  ...   xGA  \
0    1      Real Madrid  14  11  2   1  31   9   35 30.61  ... 17.99   
1    2           Girona  14  11  2   1  32  17   35 28.95  ... 21.65   
2    3  Atletico Madrid  13  10  1   2  30  12   31 28.38  ... 12.06   
3    4        Barcelona  14   9  4   1  27  14   31 28.75  ... 15.03   
4    5    Athletic Club  14   7  4   3  26  18   25 28.25  ... 17.69   
5    6    Real Sociedad  14   7  4   3  25  17   25 20.85  ... 19.71   
6    7       Real Betis  14   6  6   2  18  16   24 20.92  ... 16.78   
7    8           Getafe  14   4  7   3  17  18   19 17.61  ... 20.37   
8    9   Rayo Vallecano  14   4  7   3  16  18   19 18.51  ... 20.81   
9   10         Valencia  14   5  4   5  16  18   19 17.08  ... 16.64   
10  11       Las Palmas  14   5  3   6  11  13   18 13.09  ... 19.86   
11  12           Alaves  14   4  3   7  14  19   15 21.05  ... 19.73   
12  13       Villarreal  14   4  3   7  21  25   15 19.

In [36]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()

    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
    # convert JSON data into Python dictionary
    data = json.loads(json_data)
    
    # Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      
    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      # print('Added data for {}.'.format(team))
      
    
    for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      
    cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
    cols_to_mean = ['ppda_coef', 'oppda_coef']
    
    frames = []
    for team, df in dataframes.items():
      sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
      mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
      final_df = sum_data.join(mean_data)
      final_df['team'] = team
      final_df['matches'] = len(df)
      frames.append(final_df)

    full_stat = pd.concat(frames)
    
    full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
    full_stat.sort_values('pts', ascending=False, inplace=True)
    full_stat.reset_index(inplace=True, drop=True)
    full_stat['position'] = range(1,len(full_stat)+1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
    full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)
    
    col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
    full_stat = full_stat[col_order]
    full_stat = full_stat.set_index('position')
    # print(full_stat.head(20))
    
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()
  

team  matches  wins  draws  loses  scored  \
             position                                                         
La_Liga 2014 1               Barcelona       38    30      4      4     110   
             2             Real Madrid       38    30      2      6     118   
             3         Atletico Madrid       38    23      9      6      67   
             4                Valencia       38    22     11      5      70   
             5                 Sevilla       38    23      7      8      71   

                       missed  pts     xG  xG_diff  ...   xGA  xGA_diff  \
             position                               ...                   
La_Liga 2014 1             21   94 102.98    -7.02  ... 28.44      7.44   
             2             38   92  95.77   -22.23  ... 42.61      4.61   
             3             29   78  57.05    -9.95  ... 29.07      0.07   
             4             32   77  55.06   -14.94  ... 39.39      7.39   
             5             45   76  69.53    -1.47  ... 47.86      2.86   

                       npxGA  npxGD  ppda_coef  oppda_coef  deep  \
             position                                              
La_Liga 2014 1         24.73  73.05       5.68       16.37   489   
             2         38.89  47.21      10.21       12.93   351   
             3         26.84  25.75       8.98        9.24   197   
             4         33.45  16.26       8.71        7.87   203   
             5         41.92  20.18       8.28        9.48   305   

                       deep_allowed  xpts  xpts_diff  
             position                                 
La_Liga 2014 1                  114 94.08       0.08  
             2                  153 81.75     -10.25  
             3                  123 73.14      -4.86  
             4                  172 63.71     -13.29  
             5                  168 67.39      -8.61  

[5 rows x 21 columns]

In [37]:


data.to_csv('understat.com.csv')

